In [27]:
import pandas as pd
import os
import torch
import sqlite3
from sqlite3 import Error

def show_tables(conn):
    """ show all tables in the database """
    cur = conn.cursor()
    query = "SELECT name FROM sqlite_master WHERE type='table';"
    cur.execute(query)
    result = cur.fetchall()
    return [result[i][0] for i in range(len(result))]

def drop_temporary_tables(conn):
    """ drop temporary tables in the database """
    cur = conn.cursor()
    temporary_table_names = [table for table in show_tables(conn) if 'temporary' in table]
    for temporary_table_name in temporary_table_names:
        cur.execute(f"DROP TABLE IF EXISTS {temporary_table_name};")
    conn.commit()
    
def print_query(query,conn):
    cur = conn.cursor()
    cur.execute(query)
    result = cur.fetchall()
    for row in result:
        print(row)

def get_recording_start_stop(conn):
    """
    author: Andrew Smith
    date: 17 April 2025
    description: Get recording start and stop times from the zdb database 
    table internal_property. The table internal_property gets created the 
    first time someone opens a
    Ponemah experiment in NeuroScore. So, the recording start stop times are
    somewhere in the Ponemah files.

    future: get recording start stop directly from Ponemah files to try and
    avoid zdb
    """
    """ get recording start and stop times """
    cur = conn.cursor()
    query = "SELECT value FROM internal_property WHERE key='RecordingStart'"
    cur.execute(query)
    result = cur.fetchall()
    recording_start = int(result[0][0])
    query = "SELECT value FROM internal_property WHERE key='RecordingStop'"
    cur.execute(query)
    result = cur.fetchall()
    recording_stop = int(result[0][0])
    return recording_start, recording_stop

import torch
import os
import torch
from torch import nn
from torch.nn.functional import relu
from mne.io import read_raw_edf
from tqdm import tqdm
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

class ResidualBlock(nn.Module):
    def __init__(self,in_feature_maps,out_feature_maps,n_features) -> None:
        super().__init__()
        self.c1 = nn.Conv1d(in_feature_maps,out_feature_maps,kernel_size=8,padding='same',bias=False)
        self.bn1 = nn.LayerNorm((out_feature_maps,n_features),elementwise_affine=False)

        self.c2 = nn.Conv1d(out_feature_maps,out_feature_maps,kernel_size=5,padding='same',bias=False)
        self.bn2 = nn.LayerNorm((out_feature_maps,n_features),elementwise_affine=False)

        self.c3 = nn.Conv1d(out_feature_maps,out_feature_maps,kernel_size=3,padding='same',bias=False)
        self.bn3 = nn.LayerNorm((out_feature_maps,n_features),elementwise_affine=False)

        self.c4 = nn.Conv1d(in_feature_maps,out_feature_maps,1,padding='same',bias=False)
        self.bn4 = nn.LayerNorm((out_feature_maps,n_features),elementwise_affine=False)

    def forward(self,x):
        identity = x
        x = self.c1(x)
        x = self.bn1(x)
        x = relu(x)

        x = self.c2(x)
        x = self.bn2(x)
        x = relu(x)

        x = self.c3(x)
        x = self.bn3(x)
        x = relu(x)

        identity = self.c4(identity)
        identity = self.bn4(identity)

        x = x+identity
        x = relu(x)
        
        return x
    
class Frodo(nn.Module):
    def __init__(self,n_features) -> None:
        super().__init__()
        self.n_features = n_features
        self.block1 = ResidualBlock(1,8,n_features)
        self.block2 = ResidualBlock(8,16,n_features)
        self.block3 = ResidualBlock(16,16,n_features)

        self.gap = nn.AvgPool1d(kernel_size=n_features)
        self.fc1 = nn.Linear(in_features=16,out_features=3)
    def forward(self,x,classification=True):
        x = x.view(-1,1,self.n_features)
        x = self.block1(x)
        x = self.block2(x)
        x = self.block3(x)
        x = self.gap(x)
        if(classification):
            x = self.fc1(x.squeeze())
            return x
        else:
            return x.squeeze()
        
class Gandalf(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.encoder = Frodo(n_features=5000)
        self.lstm = nn.LSTM(16,32,bidirectional=True)
        self.fc1 = nn.Linear(64,3)
    def forward(self,x_2d,classification=True):
        x_2d = x_2d.view(-1,9,1,5000)
        x = []
        for t in range(x_2d.size(1)):
            xi = self.encoder(x_2d[:,t,:,:],classification=False)
            x.append(xi.unsqueeze(0))
        x = torch.cat(x)
        out,_ = self.lstm(x)
        if(classification):
            x = self.fc1(out[-1])
        else:
            x = out[-1]
        return x
    
class EEGDataset(torch.utils.data.Dataset):
    def __init__(self,X):
        self.len = len(X)
        self.X = torch.cat([torch.zeros(4,5000),X,torch.zeros(4,5000)])

    def __len__(self):
        return self.len

    def __getitem__(self, idx):
        return self.X[idx:idx+9].flatten()
    
def score_recording(edf_path,model_path,eeg_ch_name,device,zdb_path):
    if not os.path.exists(model_path):
        raise FileExistsError(model_path)

    model = Gandalf()
    model.load_state_dict(torch.load(model_path,map_location='cpu'))

    raw = read_raw_edf(input_fname=edf_path)
    data = raw.get_data(picks=eeg_ch_name)
    eeg = torch.from_numpy(data[0]).float()
    eeg = eeg.view(-1,5000)

    model.eval()
    model.to(device)

    with torch.no_grad():
        dataloader = DataLoader(EEGDataset(eeg),batch_size=32)
        logits = torch.cat([model(Xi.to(device)).cpu() for Xi in tqdm(dataloader)])
        y_pred = logits.softmax(dim=1).argmax(axis=1)

    pd.DataFrame(y_pred,columns=['y_pred']).to_csv(f"{zdb_path.replace('.zdb','.y_pred')}",index=False)
    # pd.DataFrame(logits).to_csv(f"{edf_path.replace('.edf','.logits')}",index=False)

In [36]:
os.system(f'rm -rf data')
source_dir = './CourtneyEEGDataJan2025'
target_dir = './data'
os.system(f'mkdir {target_dir}')

for experiment_dir in os.listdir(source_dir):
    print(experiment_dir)
    experiment_dir_path = os.path.join(source_dir,experiment_dir)
    if not os.path.exists(experiment_dir_path):
        continue
    os.system(f'mkdir \"{target_dir}/{experiment_dir}\"')
    os.system(f'cp \"{experiment_dir_path}/\"*.zdb \"{target_dir}/{experiment_dir}/\"')

os.system(f'tree data')

24-Jun PD49
24-Jun PD42
24-Jun PD28
24-Jun PD35
24-Jun PD56
data
├── 24-Jun PD28
│   ├── EKyn Sleep and Sleep Dep 24-Jun PD28.24-Jun-A1.20240730181711.zdb
│   ├── EKyn Sleep and Sleep Dep 24-Jun PD28.24-Jun-A2.20240730181711.zdb
│   ├── EKyn Sleep and Sleep Dep 24-Jun PD28.24-Jun-B1.20240730181711.zdb
│   ├── EKyn Sleep and Sleep Dep 24-Jun PD28.24-Jun-B2.20240730181711.zdb
│   ├── EKyn Sleep and Sleep Dep 24-Jun PD28.24-Jun-D1.20240730181711.zdb
│   └── EKyn Sleep and Sleep Dep 24-Jun PD28.24-Jun-D2.20240730181711.zdb
├── 24-Jun PD35
│   ├── EKyn Sleep and Sleep Dep 24-Jun PD35.24-Jun-A1.20240806200742.zdb
│   ├── EKyn Sleep and Sleep Dep 24-Jun PD35.24-Jun-A2.20240806200742.zdb
│   ├── EKyn Sleep and Sleep Dep 24-Jun PD35.24-Jun-B1.20240806200742.zdb
│   ├── EKyn Sleep and Sleep Dep 24-Jun PD35.24-Jun-B2.20240806200742.zdb
│   ├── EKyn Sleep and Sleep Dep 24-Jun PD35.24-Jun-D1.20240806200742.zdb
│   └── EKyn Sleep and Sleep Dep 24-Jun PD35.24-Jun-D2.20240806200742.zdb
├── 24-Jun PD42

0

In [37]:
"""
Sleep Staging Pipeline
"""

"""
Step 1: 
    cd aurora-sleep-staging
    mkdir data

Place each *experiment* directory in bkp AND data, we only work from data.

Always have a backup. No need to automate this. Just make it easy enough to start over.
"""


experiments_dir = f'data'
source_dir = f'./CourtneyEEGDataJan2025'
experiments = os.listdir(experiments_dir)
model_path = 'aurora-sleep-staging/gandalf.pt'
eeg_ch_name = "EEG 1"
device = 'mps'

for experiment in experiments:
    experiment_dir = os.path.join(experiments_dir, experiment)
    if not os.path.isdir(experiment_dir):
        continue
    print(f'Processing {experiment_dir}')
    recording_ids = sorted([file.replace('.zdb','') for file in os.listdir(experiment_dir) if file.endswith('.zdb')])
    for recording_id in recording_ids:
        zdb_path = os.path.join(experiment_dir, recording_id + '.zdb')
        edf_path = f'{source_dir}/{experiment}/{recording_id}.edf'
        if not os.path.exists(zdb_path):
            print(f'{zdb_path} does not exist')
            continue
        if not os.path.exists(edf_path):
            print(f'{edf_path} does not exist')
            continue
        print(f'Processing {recording_id}')
        print(experiment)
        if "24-Jun-B1" in recording_id:
            eeg_ch_name = 'EEG 2'
        else:
            eeg_ch_name = 'EEG 1'
        print(eeg_ch_name)
        score_recording(edf_path=edf_path,model_path=model_path,eeg_ch_name=eeg_ch_name,device=device,zdb_path=zdb_path)

Processing data/24-Jun PD49
Processing EKyn Sleep and Sleep Dep 24-Jun PD49.24-Jun-A1.20240820203421
24-Jun PD49
EEG 1
Extracting EDF parameters from /Users/andrew/neuroscore/CourtneyEEGDataJan2025/24-Jun PD49/EKyn Sleep and Sleep Dep 24-Jun PD49.24-Jun-A1.20240820203421.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


100%|██████████| 492/492 [00:14<00:00, 33.05it/s]

Processing EKyn Sleep and Sleep Dep 24-Jun PD49.24-Jun-A2.20240820203421
24-Jun PD49
EEG 1
Extracting EDF parameters from /Users/andrew/neuroscore/CourtneyEEGDataJan2025/24-Jun PD49/EKyn Sleep and Sleep Dep 24-Jun PD49.24-Jun-A2.20240820203421.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...



100%|██████████| 799/799 [00:23<00:00, 33.43it/s]

Processing EKyn Sleep and Sleep Dep 24-Jun PD49.24-Jun-B1.20240820203421
24-Jun PD49
EEG 2
Extracting EDF parameters from /Users/andrew/neuroscore/CourtneyEEGDataJan2025/24-Jun PD49/EKyn Sleep and Sleep Dep 24-Jun PD49.24-Jun-B1.20240820203421.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


...


100%|██████████| 798/798 [00:24<00:00, 33.07it/s]

Processing EKyn Sleep and Sleep Dep 24-Jun PD56.24-Jun-D1.20240827201830
24-Jun PD56
EEG 1
Extracting EDF parameters from /Users/andrew/neuroscore/CourtneyEEGDataJan2025/24-Jun PD56/EKyn Sleep and Sleep Dep 24-Jun PD56.24-Jun-D1.20240827201830.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...



100%|██████████| 798/798 [00:23<00:00, 33.43it/s]

Processing EKyn Sleep and Sleep Dep 24-Jun PD56.24-Jun-D2.20240827201830
24-Jun PD56
EEG 1
Extracting EDF parameters from /Users/andrew/neuroscore/CourtneyEEGDataJan2025/24-Jun PD56/EKyn Sleep and Sleep Dep 24-Jun PD56.24-Jun-D2.20240827201830.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...



100%|██████████| 798/798 [00:23<00:00, 33.42it/s]


In [38]:
experiments_dir = f'data'
experiments = os.listdir(experiments_dir)
model_path = 'aurora-sleep-staging/gandalf.pt'
device = 'mps'

for experiment in experiments:
    experiment_dir = os.path.join(experiments_dir, experiment)
    if not os.path.isdir(experiment_dir):
        continue
    print(f'Processing {experiment_dir}')
    recording_ids = sorted([file.replace('.zdb','') for file in os.listdir(experiment_dir) if file.endswith('.zdb')])
    for recording_id in recording_ids:
        zdb_path = os.path.join(experiment_dir, recording_id + '.zdb')
        if not os.path.exists(zdb_path):
            print(f'{zdb_path} does not exist')
            continue
        print(f'Processing {recording_id}')

        y_pred = pd.read_csv(f"{zdb_path.replace('.zdb','.y_pred')}")
        y_pred = pd.DataFrame(y_pred).astype('str')
        y_pred.loc[y_pred['y_pred'] == '0','y_pred'] = 'Sleep-Paradoxical'
        y_pred.loc[y_pred['y_pred'] == '1','y_pred'] = 'Sleep-SWS'
        y_pred.loc[y_pred['y_pred'] == '2','y_pred'] = 'Sleep-Wake'

        try:
            conn = sqlite3.connect(zdb_path)
        except Error as e:
            print(e)

        recording_start,recording_stop = get_recording_start_stop(conn)
        length_ns = recording_stop - recording_start # ns
        length_s = length_ns * 1e-7 # s
        hh = length_s // 3600
        mm = (length_s % 3600) // 60
        ss = ((length_s % 3600) % 60)

        print("duration:",hh,"hours",mm,"minutes",ss,"seconds")
        print("duration seconds:",length_s)
        print("recording start:",recording_start)
        print("recording stop:",recording_stop)

        # drop_temporary_tables(conn)
        print(show_tables(conn))
        cur = conn.cursor()
        cur.execute("INSERT INTO scoring_key (type) VALUES ('Automatic');")
        conn.commit()
        key_id = cur.lastrowid

        cur = conn.cursor()
        cur.execute("INSERT INTO scoring_revision (name, is_deleted, version) VALUES ('Gandalf', 0, 0);")
        conn.commit()
        scoring_revision_id = cur.lastrowid

        cur = conn.cursor()
        cur.execute(f"INSERT INTO scoring_revision_to_key (revision_id, key_id) VALUES ({scoring_revision_id}, {key_id});")
        conn.commit()

        epoch_start_time = int(int(recording_start * 1e-8) * 1e8) # decaseconds
        epoch_stop_time = epoch_start_time + int(1e8)

        for score in y_pred.values.flatten():
            query = f"""
                INSERT INTO scoring_marker 
                (starts_at, ends_at, notes, type, location, is_deleted, key_id)
                VALUES 
                ({epoch_start_time}, {epoch_stop_time}, '', '{score}', '', 0, {key_id});
                """
            
            epoch_start_time = epoch_stop_time
            epoch_stop_time = epoch_start_time + int(1e8)
            
            cur = conn.cursor()
            cur.execute(query)
            
        conn.commit()
        conn.close()

Processing data/24-Jun PD49
Processing EKyn Sleep and Sleep Dep 24-Jun PD49.24-Jun-A1.20240820203421
duration: 43.0 hours 40.0 minutes 7.0 seconds
duration seconds: 157207.0
recording start: 638597684610000000
recording stop: 638599256680000000
['internal_property', 'logging_log_entry', 'scoring_key', 'scoring_marker', 'scoring_revision', 'scoring_revision_to_key', 'scoring_revision_to_comment', 'scoring_comment', 'workspace_workspace', 'temporary_scoring_group', 'temporary_scoring_group_to_key', 'temporary_scoring_group_to_comment', 'temporary_scoring_key', 'temporary_scoring_comment']
Processing EKyn Sleep and Sleep Dep 24-Jun PD49.24-Jun-A2.20240820203421
duration: 70.0 hours 57.0 minutes 42.0 seconds
duration seconds: 255462.0
recording start: 638597684610000000
recording stop: 638600239230000000
['internal_property', 'logging_log_entry', 'scoring_key', 'scoring_marker', 'scoring_revision', 'scoring_revision_to_key', 'scoring_revision_to_comment', 'scoring_comment', 'workspace_work

In [39]:
os.system(f'tar -czvf data.tar.gz data/*/*.zdb')

a data/24-Jun PD28/EKyn Sleep and Sleep Dep 24-Jun PD28.24-Jun-A1.20240730181711.zdb
a data/24-Jun PD28/EKyn Sleep and Sleep Dep 24-Jun PD28.24-Jun-A2.20240730181711.zdb
a data/24-Jun PD28/EKyn Sleep and Sleep Dep 24-Jun PD28.24-Jun-B1.20240730181711.zdb
a data/24-Jun PD28/EKyn Sleep and Sleep Dep 24-Jun PD28.24-Jun-B2.20240730181711.zdb
a data/24-Jun PD28/EKyn Sleep and Sleep Dep 24-Jun PD28.24-Jun-D1.20240730181711.zdb
a data/24-Jun PD28/EKyn Sleep and Sleep Dep 24-Jun PD28.24-Jun-D2.20240730181711.zdb
a data/24-Jun PD35/EKyn Sleep and Sleep Dep 24-Jun PD35.24-Jun-A1.20240806200742.zdb
a data/24-Jun PD35/EKyn Sleep and Sleep Dep 24-Jun PD35.24-Jun-A2.20240806200742.zdb
a data/24-Jun PD35/EKyn Sleep and Sleep Dep 24-Jun PD35.24-Jun-B1.20240806200742.zdb
a data/24-Jun PD35/EKyn Sleep and Sleep Dep 24-Jun PD35.24-Jun-B2.20240806200742.zdb
a data/24-Jun PD35/EKyn Sleep and Sleep Dep 24-Jun PD35.24-Jun-D1.20240806200742.zdb
a data/24-Jun PD35/EKyn Sleep and Sleep Dep 24-Jun PD35.24-Jun-D2

0